In [2]:
import sys, os, time
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
# %load_ext autoreload
# %autoreload 2
# %pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
from torch.utils.data import Dataset, DataLoader
import gym
import inspect
import numpy as np
%matplotlib notebook
#%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

import project as proj

np.set_printoptions(linewidth=120)

In [4]:
x = np.random.randn(10)


In [3]:
root = os.environ['FOUNDATION_SAVE_DIR']
names = [n for n in os.listdir(root) if True]
for i, name in enumerate(names):
    print(i,name)

0 test-cmd_191121-223836
1 test-cmd_191123-013853
2 test-cmd_191123-014508
3 test-cmd_191123-104827
4 test-cmd_191123-105743
5 test-cmd_191123-110022
6 test-cmd_191123-110320
7 test-cmd_191123-110650
8 test-cmd_191123-110727
9 test-cmd_191123-111846
10 test-pycharm_191121-222639
11 test-pycharm_191121-225857
12 test-pycharm_191123-005824
13 test-pycharm_191123-013319
14 test-pycharm_191123-013427
15 test-pycharm_191123-013626
16 test-pycharm_191123-111037
17 test-pycharm_191123-111801


In [11]:
if True:
    name = names[0]
    path = os.path.join(root, name)
    A = None
    print('Using: {}'.format(name))
else:
    path = None
    A = train.get_config('model')
    print('Using: blank')

Using: test-cmd_191121-223836


In [13]:
A, (dataset, *other), model = train.load(path=path, A=A, get_model=proj.get_model, get_data=proj.get_data)
print(model)
print(model.optim)

Found 0 checkpoints. However, a config file was found
Loaded /mnt/c/Users/anwan/Documents/workspace/trained_nets/test-cmd_191121-223836
Dataset mnist moved to cpu
Dataset din=(1, 28, 28), dout=10
input_dim is an alias for din
input_dim default: (1, 28, 28)
output_dim is an alias for dout
output_dim default: 10
nonlin default: prelu
output_nonlin default: None
criterion default: cross-entropy
Model(
  (net): Sequential(
    (0): Flatten()
    (1): Linear(in_features=784, out_features=300, bias=True)
    (2): PReLU(num_parameters=1)
    (3): Linear(in_features=300, out_features=10, bias=True)
  )
  (criterion): CrossEntropyLoss()
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [14]:
A.dataset.batch_size = 64
loader = iter(train.get_loaders(dataset, batch_size=A.dataset.batch_size, num_workers=A.num_workers,
                            shuffle=True, drop_last=False, ))

In [15]:
batch = next(loader)
batch = util.to(batch, A.device)

In [16]:
X, Y = batch
X.shape, Y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))